<img src="Figures/top_ML.png" alt="Drawing" style="width: 1000px;"/>

# Sesión 4. Ejemplo de aprendizaje supervisado: clasificación.

## *Clasificación binaria de precios de electricidad en el Mercado Diario.*

**Objetivo:** predecir en qué horas el precio de la electricidad en el Mercado Diario será elevado, siendo la **clase 0** para valores menores al cuartil superior Q3, y **clase 1** para valores mayores al cuartil superior Q3 (precio electricidad > Q3 = 59 €/MWh).  Se utilizarán datos históricos de la variable target que queremos clasificar y de otros atributos (features) que pueden ayudar a predecir modelo.


**Contexto:** Una empresa que compre energía directamente en el mercado diario, quiere conocer el día anterior en que franjas horarias del dia siguiente el coste de la electricidad va a ser excesivamente elevado. Así pueden re-organizar (si es pobible) el consumo de las cargas o descargar baterías durante esas horas, reduciendo el coste de la factura eléctrica.

### Antes de empezar:

* En el archivo **S4_ejemplo.csv** se encuentra el conjunto de datos de entrada de este ejemplo (atributos + etiqueta). 


## **1. Importar librerías y datos**


In [2]:
# Importamos las librerías
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Cargamos el conjunto de datos de entrada
dataset = pd.read_csv('Data/S4_ejemplo.csv', delimiter=';')

## **2. Comprender los datos**

Es necesario visualizar y comprender los datos con los que vamos a trabajar, así como conocer sus características. 

1. ¿Cuántos datos hay? ¿Cuántos atributos hay en los datos?  
2. ¿Qué significan?
3. ¿Falta algún dato?
4. ¿Están balanceadas las etiquetas? 
4. Resumen estadístico del conjunto de datos de entrada.

**1. ¿Cuántos datos hay?**  Hay 21551 filas y 10 columnas en total.  **¿Cuántos atributos hay en los datos?** Existen 9 atrubutos y 1 etiqueta (lo que queremos clasificar).

In [31]:
# Filasxcolumnas de los datos
dataset.shape


In [1]:
# Observa las primeras 5 filas de datos
dataset.head()

In [2]:
# Observa las últimas 5 filas de datos
dataset.tail()

**2. ¿Qué significan?** 
* ***[Hora, Mes]*** Hora y mes de cada una de las observaciones. Son valores enteros *int64*.

* ***[Hidraul, Eolica, Ciclocomb, Cogener, Nuclear, Carbon]*** se refiere a la energía programada horaria en el mercado diario por tipo de producción.  Son valores reales *float*.

* ***[Demanda]*** es la totalidad de energía programada en el mercado diario eléctrico en España.  Son valores reales *float*.

* ***[Clases]*** son las etiquetas de precio que queremos predecir. Son valores enteros *int64*.




In [3]:
# Formato de los datos
dataset.dtypes

**3. ¿Falta algún dato?** Se comprueba si falta algún dato, y de ser así, se realiza el recuento de celdas vacías en cada atributo. En este caso, no falta ningún dato en el conjunto de datos de entrada (no existen valores *Nan*).

In [4]:
# Comprobar si falta algún dato y en qué atributo
dataset.isna().sum()

**4. ¿Están balanceadas las etiquetas?** Existe un desbalance de las etiquetas. La etiqueta 0 abarca un 74% de las muestras, mientras que la 1, un 26%. 

In [5]:
# Comprobar si las etiquetas están desvalanceadas
dataset['Clases'].value_counts()

**5. Resumen estadístico del conjunto de datos de entrada:** La estadística descriptiva recolecta y analiza el conjunto de datos de entrada con el objetivo de describir las características y comportamientos de este conjunto mediante las siguientes medidas resumen: número total de observaciones (count), media (mean), desviación estándar (std), valor mínimo (min), valor máximo (max) y los valores de los diferentes cuartiles (25%, 50%, 75%).

In [6]:
# Datos estadísticos de cada uno de los atributos
dataset.describe()

## **3. Visualizar los datos**

Una manera visual de entender los datos de entrada. 
1. Histograma
2. Curva de densidad
3. Boxplots
4. Matriz de correlación


**1. Histograma**

Respresentación gráfica de cada uno de los atributos en forma de barras, donde la superficie de la barra es proporcional a la frecuencia de los valores representados.

In [7]:
histograma = dataset.hist(xlabelsize=10, ylabelsize=10, bins=100, figsize=(10, 10))

**2. Gráfico de densidades**

Visualiza la distribución de los datos. Es una variable del histograma, pero elimina el ruido, por lo que son mejores para determinar la forma de distribución de un atributo. Lo spicos del gráfico de densidad ayudan a mostrar dónde los valores se concentran más. 

In [8]:
density = dataset.plot(kind='kde', x=4, subplots=True, legend=True, layout=(4, 3), figsize=(10, 10), sharex=False,
                        fontsize=8, stacked=True) 

**3. Boxplots** 

El boxplot (diagrama de caja) nos permite identificar los valores atípicos y comparar distribuciones. Además, se conoce como se distribuyen el 50% de los valores (dentro de la caja). 

En este caso, los outliers aportan bastante información, como por ejemplo un dia sin viento, que provoca un aumento considerable de la energía térmica de carbón. 

In [9]:
atributos_boxplot = dataset.plot(kind='box', subplots=True, layout=(4, 3), figsize=(11, 11), sharex=False,
                                 sharey=False, fontsize=10)

**¿Cómo ha cambiado la distribución de las fuentes de generación en el último año?** 

In [10]:
atributos_boxplot = dataset[12910:].plot(kind='box', subplots=True, layout=(4, 3), figsize=(11, 11), sharex=False,
                                 sharey=False, fontsize=10)

**4. Matriz de correlación** 

Tabla de doble entrada de los atributos

In [11]:
# Cálculo de coeficientes de correlación
corr_matrix = dataset.corr(method='pearson') 

# Matriz de correlación
fig = plt.figure(figsize=(11, 10))
ax = fig.add_subplot(111)
cax = ax.matshow(corr_matrix, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0, 10, 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(corr_matrix.columns)
ax.set_yticklabels(corr_matrix.columns)
plt.show()


## *4. Preparar los datos*

1. Limpieza de datos (data cleaning)
2. Transformación

Primero, divido los datos en **atributos**: X (features) y **etiquetas**: y (target)

In [16]:
# Atributos X (features); etiquetas y (target)
X = dataset.drop(['Clases'], axis=1) 
y = dataset['Clases']

**1. Limpieza de datos (data cleaning)**

No exisiten Nan en los datos de entrada y no se eliminarán los outliers en este ejemplo. Esto se tratará en la práctica. 



**2. Transformación**. 

Se escalan los datos utilizando el método de *MinMaxScaler()*, que escala y traduce cada atributo individualmente de tal manera que está dentro del rango [0,1]. Esto es necesario hacerlo cuando las escalas de los atributos son diferentes (eg. mes [0, 11], eólica [515.5, 18996.3])

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = X.copy()
X_scaled = pd.DataFrame(scaler.fit_transform(df_scaled))
X_scaled.columns = df_scaled.columns
X_scaled.head()

## *5. Dividir los datos*

Se utiliza la semilla ***randome_state=0*** para todos los ejercicios. ***Suffle=True*** indica que los datos se reparten de forma aleatoria entre entreno y prueba. Esto reduce la varianza y evita que el modelo no cree un sobreajuste (overfitting). 

In [18]:
from sklearn.model_selection import train_test_split

test_size = 0.2  # porcentaje de los datos de entrada que utilizaré para validar el modelo

# Divido los datos en datos de entreno, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, random_state=0,
                                                    shuffle=True)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=0,
                                                    shuffle=True)

## *6. Construcción y evaluación de modelos*

* La métrica de evaluacion seleccionada es **balanced_accuracy**. En los problemas de clasificación binaria sirve para evaluar a los conjuntos de datos desequilibrados. Se define como el promedio de **Recall** obtenido de cada clase.
* Se evaluan los siguientes algoritmos: 
    * **Logistic Regression** *LogisticRegression()*
    * **Support Vector Machine** Classifier *SVC()*
    * **K-nearest neighbors Classifier** *KNeighborsClassifier()*
    * **Decision Tree Classifier** *DecisionTreeClassifier()*

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

num_folds = 5
error_metrics = {'balanced_accuracy'}
models = {('LR', LogisticRegression()), ('SVC', SVC()), ('KNN', KNeighborsClassifier()),
           ('DTC', DecisionTreeClassifier())}

results = [] # guarda los resultados de las métricas de evaluación
names = []  # Nombre de cada algoritmo
msg = []  # imprime el resumen del método de cross-validation


Se entrenan cada uno de los modelos, se guarda su resultado y se comparan gráficamente. El que mejor resultado da es el **KNN**, ya que su balanced_accuracy es la más alta y su desviación típica es menor (más preciso).

In [13]:
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

# Entreno con validación cruzada
for scoring in error_metrics:
    print('Métrica de evaluación: ', scoring)
    for name, model in models:
        print('Modelo ', name)
        cross_validation = KFold(n_splits=num_folds, shuffle=True, random_state=0)
        cv_results = cross_val_score(model, X_train, y_train, cv=cross_validation, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        resume = (name, cv_results.mean(), cv_results.std())
        msg.append(resume)
    print(msg)

    # Comparar resultados entre algoritmos
    fig = plt.figure()
    fig.suptitle('Comparación de algoritmos con métrica de evaluación: %s' %scoring)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Modelos candidatos')
    ax.set_ylabel('%s' %scoring)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    plt.show()

    results = []


## *7. Ajustar hiperparámetros*

Pasos para realizar el hiperajuste de los parámetros:
* Especificar el modelo a ajustar: *KNeighborsClassifier()*
* Especificar una métrica para optimizar: *balanced_accuracy*
* Definir los rangos de los parámetros de búsqueda: *params*
* Asignar un método de validación: *KFold*
* Entrenar con los datos de validación: *X_val*

In [14]:
# KNeighborsRegressor

modelo = KNeighborsClassifier()
scoring='balanced_accuracy'
params = {
     'n_neighbors': [5, 20, 50], #default=5
     'weights': ['uniform','distance'], #'uniform','distance'
     'leaf_size': [30,40], #default=30
     'p': [2,6], #default=2
 }

# Búsqueda de la mejor combinación de hiperparámetros
cross_validation = KFold(n_splits=5, shuffle=True, random_state=0)
my_cv = cross_validation.split(X_val)
gsearch = GridSearchCV(estimator=modelo, param_grid=params, scoring=scoring, cv=my_cv)
gsearch.fit(X_val, y_val)

# Imprimo el mejor resultado
print("Mejor resultado: %f utilizando los siguientes hiperparámetros %s" % (gsearch.best_score_, gsearch.best_params_))
means = gsearch.cv_results_['mean_test_score']
stds = gsearch.cv_results_['std_test_score']
params = gsearch.cv_results_['params']

## *8. Evaluación final del modelo*



Métricas de evaluación:
  * 1. Matriz de confusión
  * 2. Coeficiente de Matthews (MCC)

    
Finalmente, entreno al modelo con los hiperparámetros óptimos encontrados en el apartado anterior y realizo las predicciones. 

In [24]:

modelo_final = KNeighborsClassifier(n_neighbors=5, weights='distance', leaf_size=30, p=2)
modelo_final.fit(X_train,y_train)  # Se entrena al modelo KNN
y_predict = modelo_final.predict(X_test)  # Se calculan las predicciones



**1. Matriz de confusión**

In [15]:
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix

confusion_matrix = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict))
print(confusion_matrix)

In [16]:
# Gráfico no normalizado de la martiz de confusión
disp = plot_confusion_matrix(modelo_final, X_test, y_test,
                                 cmap=plt.cm.YlGn, values_format = '')

**2. Coeficiente de Matthews (MCC)**

El MCC utiliza coeficientes de correlación entre -1 y +1. 
* Coeficiente +1 representa una predicción perfecta
* Coeficiente 0 representa una predicción media aleatoria
* Coeficiente -1 representa una predicción inversa. 

In [17]:
from sklearn.metrics import matthews_corrcoef

matthews_corrcoef(y_test, y_predict)

## 9. Conclusiones  

* El modelo KNN tiene bastante acierto, incluso en la clase minoritaria (precios de la electricidad elevados = 1).
* El atributo que más influye en la predicción es la generación de carbón. 
* Se pueden realizar mejoras, como por ejemplo feature selection (elegir atributos con mayor peso en la clasificación), entrenar con otros algoritmos (Random Forest), comprobar si los resultados dan mejor entrenando diferentes modelos para las diferentes estaciones del año, entrenar al modelo con StratifiesShuffle, etc. 
* MCC es 0.91, lo cual indica una predicción altamente exacta y precisa. 
 